<a href="https://colab.research.google.com/github/jeraldkim/DataScienceProjects/blob/main/Scraping_Anime_from_myanimelist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import regex as re
import time
import numpy as np
from google.colab import drive


In [2]:
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
"24000anime021823.csv"
path = "drive/MyDrive/24000anime021823.csv"
anime_df = pd.read_csv(path).drop(columns=["Unnamed: 0"])

In [4]:
anime_df

,Rank,Title,Link,Score
0,1.0,Fullmetal Alchemist: Brotherhood,https://myanimelist.net/anime/5114/Fullmetal_A...,9.11
1,2.0,Bleach: Sennen Kessen-hen,https://myanimelist.net/anime/41467/Bleach__Se...,9.10
2,3.0,Steins;Gate,https://myanimelist.net/anime/9253/Steins_Gate,9.08
3,4.0,Gintama°,https://myanimelist.net/anime/28977/Gintama°,9.07
4,5.0,Kaguya-sama wa Kokurasetai: Ultra Romantic,https://myanimelist.net/anime/43608/Kaguya-sam...,9.07
...,...,...,...,...
23995,NaN,Aoi Taiken,https://myanimelist.net/anime/8574/Aoi_Taiken,4.61
23996,NaN,Ikenai Boy,https://myanimelist.net/anime/17823/Ikenai_Boy,4.61
23997,NaN,Sei Michaela Gakuen Hyouryuuki II,https://myanimelist.net/anime/2240/Sei_Michael...,4.59
23998,NaN,Dream Hazard: Akuma no Program,https://myanimelist.net/anime/4178/Dream_Hazar...,4.59


In [ ]:
anime_list = []
num_anime = 480 #ex: 5 means 5*50 we want top 250 anime 0 would mean only top 50 anime so not run loop 200 = top 10000 anime
n = 0

while n < num_anime:
    if n == 0:
      curr_link = "https://myanimelist.net/topanime.php"
    elif n > 0:
      curr_link = "https://myanimelist.net/topanime.php?limit=" + str(n * 50)
    n += 1

    response = requests.get(curr_link)
    html_content = response.content
    soup = BeautifulSoup(html_content, "html.parser")
    table = soup.find("table", class_="top-ranking-table")
    time.sleep(3) #necessary condition to prevent site from blocking your requests


    if table == None:
      print(curr_link)
      print(soup)
      bug1 = response
      bug2 = html_content
      bug3 = soup
    
    for row in table.find_all("tr")[1:51]:
      anime = {}
      rank = row.find("td", class_="rank").text.strip()
      if rank == '-':
        anime["Rank"] = None
      else:
        anime["Rank"] = int(rank)
      anime["Title"] = row.find("div", class_="di-ib clearfix").find("a").text
      anime["Link"] = row.find("a").get("href")
      score = row.find("td", class_="score").text.strip()
      if score == 'N/A':
        anime["Score"] = None
      else:
        anime["Score"] = float(score)
      anime_list.append(anime)

anime_df = pd.DataFrame(anime_list)

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/My Drive/24000anime021823.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  anime_df.to_csv(f)

#method to save df to csv file on google drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# real = anime_df["Link"] #for generating entire data

# tests = anime_scraped_df["Link"][:5] #use this one to test for implementation
new_anime_list = []
#downloaded = anime_scraped_df

curr = 1 #change these for where you stopped off
n = 1000

while n <= 24000:
  new_anime_list = []

  for link in anime_df[curr-1:n]["Link"]: #changed curr-1 to include 0 index


# for link in tests: #change here for testing

      response = requests.get(link)
      html_content = response.content
      soup = BeautifulSoup(html_content, "html.parser")
      time.sleep(3)

      anime = {}
      anime_type = soup.find("span",class_ ="dark_text",string="Type:")
      if anime_type == None:
        anime["Type"] = None
      else:
        anime["Type"] = anime_type.find_parent().text.replace('Type:','').strip() #Type of Show
      num_episodes = soup.find("span",class_ ="dark_text",string="Episodes:")
      if num_episodes == None:
        anime["# Episodes"] = None
      else:
        anime["# Episodes"] = num_episodes.next_sibling.strip() #Number of Episodes
    
      source = soup.find("span",class_ ="dark_text",string="Source:")
      if source == None:
        anime["Source"] = None
      else:
        anime["Source"] = source.next_sibling.strip() #Source

      status = soup.find("span",class_ ="dark_text",string="Status:")
      if status == None:
        anime["Status"] = None
      else:
        anime["Status"] = status.next_sibling.strip()

      premiered = soup.find("span",class_ ="dark_text",string="Premiered:")
      if premiered == None:
        anime["Premiered"] = None
      else:
        anime["Premiered"] = premiered.find_parent().find("a").text #Season of Anime and Year

    
      aired_date = soup.find("span",class_ ="dark_text",string="Aired:")
      if aired_date == None:
        anime["Aired Date"] = None
      else:
        anime["Aired Date"] = aired_date.next_sibling.strip() #Aired Date
    
      studio = soup.find("span",class_ ="dark_text",string="Studios:")
      if studio == None:
        anime["Studios"] = None
      else: 
        anime["Studios"] = re.sub('\nStudios:\n','',studio.find_parent().text).strip() #Studio RECENTLy made a regex change for animes with more than 1 studio (rare though)

      has_genre = soup.find("span",class_ ="dark_text",string=re.compile(r'Genres:|Genre:')) #FIXED to scrape multiple genres
      if has_genre == None:
        anime["Genres"] = None
      else:
        genres = has_genre.find_parent().find_all("a") #Genre
        genre_list = ""
        for genre in genres: #add as a string
          genre_list += genre.text.strip() + ","
        anime["Genres"] = genre_list
      
      has_theme = soup.find("span",class_ ="dark_text",string=re.compile(r'Themes:|Theme:')) #NEW addition checks for Themes AND Theme
      if has_theme == None:
        anime["Themes"] = None
      else:
        themes = has_theme.find_parent().find_all("a")
        themes_list = ""
        for theme in themes:
          themes_list += theme.text.strip() + ","
        anime["Themes"] = themes_list


      demo = soup.find("span",class_ ="dark_text",string="Demographic:")
      if demo == None:
        anime["Demographic"] = None
      else:
        anime["Demographic"] = demo.find_parent().find("a").text #Demographic
    
      duration = soup.find("span",class_ ="dark_text",string="Duration:")
      if duration == None:
        anime["Duration"] = None
      else:
        anime["Duration"] = duration.next_sibling.strip() #Duration

      age_rating = soup.find("span",class_ ="dark_text",string="Rating:")
      if age_rating == None:
        anime["Age Rating"] = None
      else:
        anime["Age Rating"] = age_rating.next_sibling.strip() #Age Rating

      #New feature additions started from this line
      num_raters = soup.find("span",class_ ="dark_text",string="Score:")
      if num_raters == None:
        anime["Number of Raters"] = None
      else:
        anime["Number of Raters"] = num_raters.find_parent().find(itemprop="ratingCount").text #Number of people who rated the scores

      popularity = soup.find("span",class_ ="dark_text",string="Popularity:")
      if popularity == None:
        anime["Popularity"] = None
      else:
        anime["Popularity"] = popularity.next_sibling.strip() #Popularity Rank determined by number of "members"

      members = soup.find("span",class_ ="dark_text",string="Members:")
      if members == None:
        anime["Members"] = None
      else:
        anime["Members"] = members.next_sibling.strip() # Number of members = num of  people who added the series to the list


      favorites = soup.find("span",class_ ="dark_text",string="Favorites:")
      if favorites == None:
        anime["Favorites"] = None
      else:
        anime["Favorites"] = favorites.next_sibling.strip() #Favorites


      #Probably Final Addition of Other Related Animes


      #to include Features:

      #Adapation useful for comparing manga scores
      #Sequel
      #Prequel
      has_related = soup.find("table",class_ ="anime_detail_related_anime")
      if has_related == None:
        anime["Adaptation"] = None
        anime["Sequel"] = None
        anime["Prequel"] = None
      else:
        has_adaptation = has_related.find(string="Adaptation:")
        if has_adaptation == None:
          anime["Adaptation"] = None
        else:
          anime["Adaptation"] = has_adaptation.find_parent().next_sibling.text

        has_sequel = has_related.find(string="Sequel:")
        if has_sequel == None:
          anime["Sequel"] = None
        else:
          anime["Sequel"] = has_sequel.find_parent().next_sibling.text

        has_prequel = has_related.find(string="Prequel:")
        if has_prequel == None:
          anime["Prequel"] = None
        else:
          anime["Prequel"] = has_prequel.find_parent().next_sibling.text

      #new addition of voice actor, character, role

      Character = []
      Role = []
      Voice_Actors = []

      char_role_voice = soup.find("div",class_="detail-characters-list clearfix")

      if char_role_voice == None:
        anime["Characters"] = None
        anime["Role"] = None
        anime["Voice Actors"] = None
      else:    
        for child in char_role_voice.children:
        
          characters1 = child.find_all("h3")
          if characters1 == None:
            anime["Characters"] = None
          else:
            for chars in characters1:
              Character += [chars.text]
        
          roles1 = child.find_all("div",class_ = "spaceit_pad")
          if roles1 == None:
            anime["Role"] = None
          else:
            for chars in roles1:
              Role += [chars.text.strip()]

          va1 = child.find_all("td",class_ = "va-t ar pl4 pr4")
          if va1 == None:
            anime["Voice Actors"] = None
          else:
            for chars in va1:
              Voice_Actors += [re.sub('\\n.*','',chars.text.strip())]

      anime["Characters"] = Character
      anime["Role"] = Role
      anime["Voice Actors"] = Voice_Actors   
    
      #addition of music in anime
      opening_songs = []
      opening_artists = []
      ending_songs = []
      ending_artists = []

      has_openings = soup.find("div",class_="theme-songs js-theme-songs opnening")
      if has_openings == None:
        anime["Openings"] = None
        anime["Opening Artists"] = None
      else:
        # openings1 = has_openings.find_all("span",class_="theme-song-title")
        opening_songs = re.findall("\"(.+)\"\sby",has_openings.text)
        # for song in openings1:
        #   opening_songs += [re.sub(r'\'^\'|\'$|^\"|\"$',"",song.text)]

        opening_artists1 = has_openings.find_all("span",class_="theme-song-artist")
        for artist in opening_artists1:
          opening_artists += [re.sub("\sby\s","",artist.text)]
    
        anime["Openings"] = opening_songs
        anime["Opening Artists"] = opening_artists

      has_endings = soup.find("div",class_="theme-songs js-theme-songs ending")
      if has_endings == None:
        anime["Endings"] = None
        anime["Ending Artists"] = None
      else:
        # endings1 = has_endings.find_all("span",class_="theme-song-title")
        ending_songs = re.findall("\"(.+)\"\sby",has_endings.text)
        # for song in endings1:
        #   ending_songs += [re.sub(r'\'^\'|\'$|^\"|\"$',"",song.text)]

        ending_artists1 = has_endings.find_all("span",class_="theme-song-artist")
        for artist in ending_artists1:
          ending_artists += [re.sub("\sby\s","",artist.text)]

        anime["Endings"] = ending_songs
        anime["Ending Artists"] = ending_artists
    
      #Final Step
      new_anime_list.append(anime)    

  new_anime_df = pd.DataFrame(new_anime_list)

  drive.mount('/content/drive')

  path = '/content/drive/My Drive/' + str(curr) + "to" + str(n) + 'anime_stats_021723.csv'

  with open(path, 'w', encoding = 'utf-8-sig') as f:
    new_anime_df.to_csv(f)


  n += 1000
  curr += 1000

In [33]:
# new_anime_df = pd.DataFrame(new_anime_list)
new_anime_df.max_columns = 30
new_anime_df

,Type,# Episodes,Source,Status,Premiered,Aired Date,Studios,Genres,Themes,Demographic,...,Adaptation,Sequel,prequel,Characters,Role,Voice Actors,Openings,Opening Artists,Endings,Ending Artists
0,TV,64,Manga,Finished Airing,Spring 2009,"Apr 5, 2009 to Jul 4, 2010",Bones,"Action,Adventure,Drama,Fantasy,","Military,",Shounen,...,Fullmetal Alchemist,None,None,"[Elric, Edward, Elric, Alphonse, Mustang, Roy,...","[Main, Main, Supporting, Supporting, Supportin...","[Park, Romi, Kugimiya, Rie, Miki, Shinichiro, ...","[again, Hologram (ホログラム), Golden Time Lover (ゴ...","[YUI, NICO Touches the Walls, Sukima Switch, C...","[Uso (嘘), LET IT OUT, Tsunaida Te (つないだ手), Shu...","[SID, Miho Fukuhara, Lil'B, SCANDAL, Nakagawa ..."
1,TV,13,Manga,Finished Airing,Fall 2022,"Oct 11, 2022 to Dec 27, 2022",Pierrot,"Action,Adventure,Fantasy,",None,Shounen,...,Bleach,Bleach: Sennen Kessen-hen - Ketsubetsu-tan,Bleach,"[Kurosaki, Ichigo, Kuchiki, Rukia, Inoue, Orih...","[Main, Main, Main, Main, Main, Main, Supportin...","[Morita, Masakazu, Orikasa, Fumiko, Matsuoka, ...",[スカー],[キタニタツヤ],"[Rapport, 最果て, Number One]","[キタニタツヤ, SennaRin, Shiro Sagisu, Hazel Fernandes]"
2,TV,24,Visual novel,Finished Airing,Spring 2011,"Apr 6, 2011 to Sep 14, 2011",White Fox,"Drama,Sci-Fi,Suspense,","Psychological,Time Travel,",None,...,Steins;Gate,Steins;Gate: Oukoubakko no Poriomania,None,"[Okabe, Rintarou, Makise, Kurisu, Shiina, Mayu...","[Main, Main, Main, Main, Supporting, Supportin...","[Miyano, Mamoru, Imai, Asami, Hanazawa, Kana, ...",[Hacking to the Gate],[Kanako Itou],"[Toki Tsukasadoru Juuni no Meiyaku (刻司ル十二ノ盟約),...","[Yui Sakakibara, Takeshi Abo, Kanako Itou, Kan..."
3,TV,51,Manga,Finished Airing,Spring 2015,"Apr 8, 2015 to Mar 30, 2016",Bandai Namco Pictures,"Action,Comedy,Sci-Fi,","Gag Humor,Historical,Parody,Samurai,",Shounen,...,Gintama,Gintama.,Gintama Movie 2: Kanketsu-hen - Yorozuya yo Ei...,"[Sakata, Gintoki, Kagura, Shimura, Shinpachi, ...","[Main, Main, Main, Supporting, Supporting, Sup...","[Sugita, Tomokazu, Kugimiya, Rie, Sakaguchi, D...","[DAY×DAY, Pride Kakumei (プライド革命), Saigo made I...","[BLUE ENCOUNT, CHiCO with HoneyWorks, Aqua Tim...","[DESTINY, Saigo made II (最後までⅡ), Pride Kakumei...","[Negoto, Aqua Timez, CHiCO with HoneyWorks, TH..."
4,TV,13,Manga,Finished Airing,Spring 2022,"Apr 9, 2022 to Jun 25, 2022",A-1 Pictures,"Comedy,Romance,","School,",Seinen,...,Kaguya-sama wa Kokurasetai: Tensai-tachi no Re...,Kaguya-sama wa Kokurasetai: First Kiss wa Owar...,Kaguya-sama wa Kokurasetai? Tensai-tachi no Re...,"[Shinomiya, Kaguya, Ishigami, Yuu, Fujiwara, C...","[Main, Main, Main, Main, Main, Supporting, Sup...","[Koga, Aoi, Suzuki, Ryouta, Kohara, Konomi, Fu...",[GIRI GIRI feat Suu (GIRI GIRI feat. すぅ)],[Masayuki Suzuki],"[GIRI GIRI feat Suu (GIRI GIRI feat. すぅ), Hear...","[Masayuki Suzuki, Airi Suzuki, Miyuki Shirogan..."
